In [ ]:
from src import SharePricePrediction, PredictValues, FindValues

In [ ]:
def getFundamentals(c_name, future_years):
    predicted_values = PredictValues(c_name=c_name)
    fundamental_values = predicted_values.getFutureValues(future_year=future_years)
    print(fundamental_values)
    return (fundamental_values)

In [ ]:
fundametal_values = getFundamentals('ITC', 5)

In [ ]:
for val in fundametal_values:
    print(f'{val} : {fundametal_values[val]}')

In [ ]:
test_rev = fundametal_values['future_revenue']

test_rev_arr = [round(float(val), 2) for val in test_rev]
print(test_rev_arr)

In [ ]:
import json  # Import JSON module to handle JSON strings

def getSharePrice(c_name, future_years):
    share_price_prediction = SharePricePrediction(company_name=c_name)
    share_price = share_price_prediction.SharePrice(future_years=future_years)
    previous_share_price = share_price['previous_share_price']
    future_share_price = share_price['future_share_price']
    
    # Parse JSON strings to Python dictionaries
    data = {
        'previous_share_price': json.loads(previous_share_price.to_json(orient='records', date_format='iso')),
        'future_share_price': json.loads(future_share_price.to_json(orient='records', date_format='iso'))
    }
    return data

In [ ]:
share_price_data = getSharePrice('ITC', 5)

In [ ]:
val = share_price_data['previous_share_price']

In [ ]:
print(val[0])

In [2]:
from src import CompanyDetails, FindValues

In [ ]:
c_name = 'ITC'
details = FindValues(c_name)

In [4]:
values = details.getCompanyDetails()

for val in values:
    print(f'{val} : {values[val]}')

years : [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
revenue : [16517.97, 20272.51, 23702.62, 23726.32, 29119.31, 34046.3, 34080.35, 41826.81, 48903.91, 48952.81, 60081.36, 70245.22, 70315.49]
income : [3503.42, 4057.66, 5109.0, 5446.19, 6307.78, 7942.13, 8466.31, 9805.68, 12346.33, 13161.19, 15242.66, 19191.66, 20458.78]
eps : [2.8, 3.24, 4.08, 4.35, 5.04, 6.35, 6.77, 7.84, 9.87, 10.52, 12.18, 15.34, 16.35]
pe : 24
roe : [5.07, 5.87, 7.39, 7.88, 9.12, 11.48, 12.24, 14.18, 17.85, 19.03, 22.04, 27.75, 29.58]
operating_expence : [13116.52, 14439.47, 16888.0, 18196.92]
profit_margin : [21.21, 20.02, 21.55, 22.95, 21.66, 23.33, 24.84, 23.44, 25.25, 26.89, 25.37, 27.32, 29.1]
shareholders_equity : [60347.34, 62455.57, 69155.26, 74507.0]
outstanding_shares : 1251.41
line_data : {'support1': 395.22, 'support2': 380.68, 'support3': 355.63, 'resistance1': 420.27, 'resistance2': 430.78, 'resistance3': 455.83}
indicator_data : {'summary': 'NEUTRAL', 'rsi': 'NEUTR

In [5]:
import json
details = CompanyDetails(c_name)
share_price = details.sharePriceRange(period='2y')


share_price_json = json.loads(share_price.to_json(orient='records', date_format='iso'))
for val in share_price_json:
    print(val)

{'Date': '2023-04-05', 'Close': 362.82, 'ema100': 362.82, 'ema200': 362.82}
{'Date': '2023-04-06', 'Close': 363.71, 'ema100': 362.8376237624, 'ema200': 362.8288557214}
{'Date': '2023-04-10', 'Close': 364.84, 'ema100': 362.877274777, 'ema200': 362.8488671072}
{'Date': '2023-04-11', 'Close': 371.83, 'ema100': 363.0545564646, 'ema200': 362.9382316136}
{'Date': '2023-04-12', 'Close': 369.53, 'ema100': 363.1827830692, 'ema200': 363.0038213488}
{'Date': '2023-04-13', 'Close': 371.46, 'ema100': 363.346688355, 'ema200': 363.0879624299}
{'Date': '2023-04-17', 'Close': 375.73, 'ema100': 363.591902447, 'ema200': 363.2137538485}
{'Date': '2023-04-18', 'Close': 374.18, 'ema100': 363.801567745, 'ema200': 363.3228707257}
{'Date': '2023-04-19', 'Close': 374.42, 'ema100': 364.0118337303, 'ema200': 363.4332899224}
{'Date': '2023-04-20', 'Close': 375.87, 'ema100': 364.246648904, 'ema200': 363.5570382814}
{'Date': '2023-04-21', 'Close': 383.34, 'ema100': 364.6247350643, 'ema200': 363.7538836717}
{'Date': 